Medium article
https://pitipat-both.medium.com/detecting-ai-generated-essays-d143b1b574ef

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install accelerate -U

In [ ]:
import transformers
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Deep learning/train_essays.csv')

train_data.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [ ]:
train_data['generated'].value_counts()

0    1375
1       3
Name: generated, dtype: int64

There are only 3 AI generated essay in the provided training data.

We should get additional training data set that is AI generated

I will use this data set https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset
for additional training data

In [ ]:
train_daigt = pd.read_csv('/content/drive/MyDrive/Deep learning/train_v2_drcat_02.csv')
train_daigt.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [ ]:
train_daigt.source.value_counts()

persuade_corpus                       25996
mistral7binstruct_v2                   2421
chat_gpt_moth                          2421
llama2_chat                            2421
mistral7binstruct_v1                   2421
kingki19_palm                          1384
train_essays                           1378
llama_70b_v1                           1172
falcon_180b_v1                         1055
darragh_claude_v6                      1000
darragh_claude_v7                      1000
radek_500                               500
NousResearch/Llama-2-7b-chat-hf         400
mistralai/Mistral-7B-Instruct-v0.1      400
cohere-command                          350
palm-text-bison1                        349
radekgpt4                               200
Name: source, dtype: int64

This data set already include the training data we had before (train_essays), so we are going to use only this data set.

In [ ]:
train_daigt.label.value_counts()

0    27371
1    17497
Name: label, dtype: int64

Data Cleaning:
Remove a character that isn't in human essay

In [ ]:
human = train_daigt[train_daigt['source'] == 'persuade_corpus'].sample(n=6000) #Select only 6000 samples to reduce training time
non_human = train_daigt[train_daigt['source'] != 'persuade_corpus']

In [ ]:
char_human = set(list(''.join(human.text.to_list())))
char_non_human = set(list(''.join(non_human.text.to_list())))

In [ ]:
chars_to_remove = ''.join([x for x in char_non_human if x not in char_human])
print(chars_to_remove)

😩者🛫🧬🍰🌠安🥯🔬🤛🎃的注🍲📹🌏🦐🕹🚌👧🏙🥟🐻りá🔮😌🐸🥭🐴と🗣🚴️ê⚽🍁💨💕👩📦п📅🍔🚪🐦👪‘🌅🌴е🤓🧭🇵🐒🦁😱😴…🔥°🚚🙈🎩👨ü👦🏻🙀部⏰🚣🧦😅🥤😷🛣せ🤤🏠🌮和🍄🤩🏃”🍳🐰🚭😎😒🌈🥖こ🧡ん🍭🎵🤪🧑🛍💆🐧🧙🇧🏔📊🏊🥕🇷🥦🐢💃😈😉应📺🧐🌯🇯🌿🚔🌨□🎾📱🏄🐠🎓💖💯😝路🔜😂🏳🕰🤦驾💦🐕👮🍜🤘🐬🍗👬🎭💚🥑🏽😵🏞📰🐆🌄🤕🍝©–💫🎠💊😊🥪🍎🛀🌭💭🍕🌽🍽🎥🇸^🏀🧩🏢💧司😠用📞😕♀🤞该​👀🛠🌐🐝💥🌃响🍣🍞🤝🤟🌱🦎は🤣🚂🚫🔭す🧚🥜在💼ま🙌🧘í🏋😹🏛📸🏫🥘🎶🌳🏏🍴😨🥛📷🎉о驶─🐳👥必🇺🧽🌻🙅😔🌊止💬🛬🍮💇🌎🌞💪👌👫❄🥳💉🎅🎤🚨🙋🌌💔手🤫取😃🙊禁合🐾👇😳🚀有💤😖😓😜机🥁ā🏕😢😤💘🕺👂🌫♂🥩集🎢🍋—😘🚗🍖🧀🐶🇫🔑🎯🎄🎊时须🏟☀😲🛋🍓意护÷🍿将🎣🐭法Éд‍📣み🏨🤗😆🏯🕒🇪🏥全あ完🎸影🌸う🤢📉


In [ ]:
translation_table = str.maketrans('', '', chars_to_remove)
def remove_chars(s):
    return s.translate(translation_table)
non_human['text'] = non_human['text'].apply(remove_chars)

<ipython-input-12-bf4f8be6daca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_human['text'] = non_human['text'].apply(remove_chars)


In [ ]:
train_daigt = pd.concat([human, non_human]).reset_index(drop=True)
train_daigt.source.value_counts()

persuade_corpus                       6000
mistral7binstruct_v2                  2421
chat_gpt_moth                         2421
llama2_chat                           2421
mistral7binstruct_v1                  2421
kingki19_palm                         1384
train_essays                          1378
llama_70b_v1                          1172
falcon_180b_v1                        1055
darragh_claude_v6                     1000
darragh_claude_v7                     1000
radek_500                              500
NousResearch/Llama-2-7b-chat-hf        400
mistralai/Mistral-7B-Instruct-v0.1     400
cohere-command                         350
palm-text-bison1                       349
radekgpt4                              200
Name: source, dtype: int64

In [ ]:
train = train_daigt.sample(frac=0.8)
validation = train_daigt.drop(train.index)

In [ ]:
print(len(train))
print(len(validation))

19898
4974


I will use this pretrained model from hugging face
https://huggingface.co/microsoft/deberta-v3-small


In [ ]:
model_checkpoint = "microsoft/deberta-v3-xsmall"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Preprocess the data and turn it into tokens

In [ ]:
train = Dataset.from_pandas(train)
validation = Dataset.from_pandas(validation)

In [ ]:
train_token = train.map(preprocess_function, batched=True)
validation_token = validation.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Hyperparameters
metric_name = "roc_auc"
model_name = "deberta-v3-xsmall"
train_batch_size = 2
eval_batch_size = 8
grad_acc = 4

num_steps = len(train) // (train_batch_size * grad_acc)

args = TrainingArguments(
    "deberta-v3-xsmall-finetuned_1",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = num_steps,
    save_steps = num_steps,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    report_to='none'
)

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset = train_token,
    eval_dataset = validation_token,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Roc Auc
829,0.111800,0.059573,0.999035
1658,0.030300,0.136364,0.997817
2487,0.025300,0.064314,0.999415


TrainOutput(global_step=2487, training_loss=0.04591831127514933, metrics={'train_runtime': 1852.3493, 'train_samples_per_second': 10.742, 'train_steps_per_second': 1.343, 'total_flos': 1292746907393832.0, 'train_loss': 0.04591831127514933, 'epoch': 1.0})

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Deep learning/test_essays.csv")
test = Dataset.from_pandas(test)
test_token = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
test_preds = trainer.predict(test_token)

In [ ]:
trainer

In [ ]:
logits = test_preds.predictions
probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
sub = pd.DataFrame()
sub['id'] = test['id']
sub['generated'] = probs[:,1]
sub.to_csv('submission.csv', index=False)
sub.head()

,id,generated
0,0000aaaa,0.671467
1,1111bbbb,0.997959
2,2222cccc,0.998231


In [ ]:
# Save model
save_dir = "/content/drive/MyDrive/Deep learning/model_1"
trainer.save_model(save_dir)

In [ ]:
load_model = AutoModelForSequenceClassification.from_pretrained(save_dir)

load_trainer = Trainer(
    load_model,
    args,
    train_dataset = train_token,
    eval_dataset = validation_token,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
load_preds = load_trainer.predict(test_token)

In [ ]:
logits = load_preds.predictions
probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
sub = pd.DataFrame()
sub['id'] = test['id']
sub['generated'] = probs[:,1]
sub.to_csv('/content/drive/MyDrive/Deep learning/submission.csv', index=False)
sub.head()

,id,generated
0,0000aaaa,0.671467
1,1111bbbb,0.997959
2,2222cccc,0.998231
